In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score

# append the filepath to where torch is installed
sys.path.append('/home/millerm/.local/lib/python3.10/site-packages')
# sys.path.append('/home/username/.local/lib/python3.10/site-packages')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Load and Preprocess Data

In [2]:
file_name = "ml4h_data/project2/project2_TS_input/ptbdb_train.csv"
df_train = pd.read_csv(file_name,header=None)
x_train = df_train.iloc[:, df_train.columns != 187]
x_train = x_train.values.reshape(-1, 1, 187)
train_target = df_train.iloc[:, 187]
train_target = train_target.values
train_target

array([1., 1., 0., ..., 1., 1., 1.])

In [3]:
file_name = "ml4h_data/project2/project2_TS_input/ptbdb_test.csv"
df_test = pd.read_csv(file_name,header=None)
x_test = df_test.iloc[:, df_test.columns != 187]
x_test = x_test.values.reshape(-1, 1, 187)
test_target = df_test.iloc[:, 187]
test_target = test_target.values
test_target

array([0., 1., 0., ..., 1., 1., 0.])

In [4]:
from setsloaders import create_datasets, create_loaders

In [5]:
device = torch.device('cuda')
device

device(type='cuda')

In [6]:
datasets = create_datasets(x_train, x_test, train_target, test_target, seed=123)
trn_dl, val_dl, tst_dl = create_loaders(datasets, bs=256)

## Vanilla CNN

In [7]:
input_channels = 1
num_classes = 2
output_size = 2
train_loader = trn_dl
test_loader = tst_dl
num_epochs = 30
learning_rate = 0.005

In [8]:
class cnn(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3)
        self.fc1 = nn.Linear(5376, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = cnn(input_channels, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch}, Loss: {loss}")
        
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Accuracy: {:.2f}%'.format(accuracy))

Epoch: 0, Loss: 0.42196783423423767
Epoch: 1, Loss: 0.26996707916259766
Epoch: 2, Loss: 0.3202950954437256
Epoch: 3, Loss: 0.26360201835632324
Epoch: 4, Loss: 0.2283637672662735
Epoch: 5, Loss: 0.1429937779903412
Epoch: 6, Loss: 0.14124779403209686
Epoch: 7, Loss: 0.10368941724300385
Epoch: 8, Loss: 0.08037424087524414
Epoch: 9, Loss: 0.07947560399770737
Epoch: 10, Loss: 0.15870201587677002
Epoch: 11, Loss: 0.10222574323415756
Epoch: 12, Loss: 0.07381618767976761
Epoch: 13, Loss: 0.07818073779344559
Epoch: 14, Loss: 0.04714014753699303
Epoch: 15, Loss: 0.0847485288977623
Epoch: 16, Loss: 0.034282080829143524
Epoch: 17, Loss: 0.05068464204668999
Epoch: 18, Loss: 0.07417630404233932
Epoch: 19, Loss: 0.03859033063054085
Epoch: 20, Loss: 0.03321503847837448
Epoch: 21, Loss: 0.01942056231200695
Epoch: 22, Loss: 0.0347738116979599
Epoch: 23, Loss: 0.041800256818532944
Epoch: 24, Loss: 0.020079998299479485
Epoch: 25, Loss: 0.023609157651662827
Epoch: 26, Loss: 0.0044312928803265095
Epoch: 27,

In [9]:
model.eval()
predictions = []
ground_truth = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        ground_truth.extend(labels.cpu().numpy())

predictions = np.array(predictions)
ground_truth = np.array(ground_truth)

precision = precision_score(ground_truth, predictions, average='binary')
recall = recall_score(ground_truth, predictions, average='binary')
f1 = (2*precision*recall)/(precision+recall)

print('Precision: {:.2f}'.format(precision))
print('Recall: {:.2f}'.format(recall))
print('F1-Score: {:.2f}'.format(f1))

Precision: 0.99
Recall: 0.99
F1-Score: 0.99


In [10]:
torch.save(model, "models/vanillacnn.pth")

## Residual Blocks

In [11]:
input_channels = 1
num_classes = 2
train_loader = trn_dl
test_loader = tst_dl
num_epochs = 30
learning_rate = 0.005

In [12]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=1)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.conv1(x))
        out = self.conv2(out)
        out += residual  # Residual connection
        out = self.relu(out)
        return out

class rescnn(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(rescnn, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3)
        self.residual_block1 = ResidualBlock(64, 64, kernel_size=3)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3)
        self.residual_block2 = ResidualBlock(128, 128, kernel_size=3)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3)
        self.residual_block3 = ResidualBlock(256, 256, kernel_size=3)
        self.pool3 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(5376, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.residual_block1(x)
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.residual_block2(x)
        x = self.pool2(x)
        x = self.relu(self.conv3(x))
        x = self.residual_block3(x)
        x = self.pool3(x)
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [13]:
model = rescnn(input_channels, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch}, Loss: {loss}")
        
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print('Accuracy: {:.2f}%'.format(accuracy))

Epoch: 0, Loss: 0.3714563846588135
Epoch: 1, Loss: 0.2989402115345001
Epoch: 2, Loss: 0.31668052077293396
Epoch: 3, Loss: 0.1769210547208786
Epoch: 4, Loss: 0.2206125557422638
Epoch: 5, Loss: 0.19620147347450256
Epoch: 6, Loss: 0.1541152447462082
Epoch: 7, Loss: 0.10125810652971268
Epoch: 8, Loss: 0.08323320001363754
Epoch: 9, Loss: 0.030131559818983078
Epoch: 10, Loss: 0.09060701727867126
Epoch: 11, Loss: 0.10514943301677704
Epoch: 12, Loss: 0.029458578675985336
Epoch: 13, Loss: 0.06162691116333008
Epoch: 14, Loss: 0.04238803684711456
Epoch: 15, Loss: 0.006754663307219744
Epoch: 16, Loss: 0.016100751236081123
Epoch: 17, Loss: 0.009632227011024952
Epoch: 18, Loss: 0.017751645296812057
Epoch: 19, Loss: 0.0230732224881649
Epoch: 20, Loss: 0.007024645805358887
Epoch: 21, Loss: 0.04222329333424568
Epoch: 22, Loss: 0.010342000983655453
Epoch: 23, Loss: 0.06625109910964966
Epoch: 24, Loss: 0.01555738877505064
Epoch: 25, Loss: 0.010984146036207676
Epoch: 26, Loss: 0.0012418749975040555
Epoch:

In [14]:
model.eval()
predictions = []
ground_truth = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        ground_truth.extend(labels.cpu().numpy())

predictions = np.array(predictions)
ground_truth = np.array(ground_truth)

precision = precision_score(ground_truth, predictions, average='binary')
recall = recall_score(ground_truth, predictions, average='binary')
f1 = (2*precision*recall)/(precision+recall)

print('Precision: {:.2f}'.format(precision))
print('Recall: {:.2f}'.format(recall))
print('F1-Score: {:.2f}'.format(f1))

Precision: 0.99
Recall: 0.99
F1-Score: 0.99


In [15]:
torch.save(model, "models/residualcnn.pth")